In [1]:
import pandas as pd
import requests
import time

/home/thucdx/.local/lib/python2.7/site-packages/requests/__init__.py:80: RequestsDependencyWarning: urllib3 (1.22) or chardet (2.3.0) doesn't match a supported version!
  RequestsDependencyWarning)


In [54]:
import sys,os,os.path
# os.environ['HTTP_PROXY']="http://proxy.example.com:80"
# os.environ['HTTPS_PROXY']="https://proxy.example.com:443"
os.environ['NO_PROXY']="10.*, 192.*, localhost"


In [55]:
gps_inp = "../../samples/data/atuanhm_2705_gps_processed.csv"
vehicle_inp = "../../samples/data/atuanhm_2705_report_vehicle_processed.csv"

id = "sample car 1"
delay=0.05

In [56]:
tracking_gps_url = "http://localhost:1234/trackMany"
tracking_vehicle_url = "http://localhost:1234/track"

In [57]:
import json

def to_json(row, id=None):
    obj = {}
    obj['time'] = row['positionTime'] * 1000
    obj['lat'] = row['lat']
    obj['lng'] = row['lng']
    obj['azimuth'] = row['heading']
    if 'speed' in row:
        obj['velocity'] = row['speed']
    if 'height' in row:
        obj['height'] = row['height']
    if id is not None:
        obj["id"] = id
        
    return json.dumps(obj)

In [58]:
def merge_gps(series, id=None):
    res = []
    for u in range(series.shape[0]):
        res.append(json.loads(series.iloc[u]['json']))
        
    obj = {"samples": res}
    if id is not None:
        obj["id"] = id
    
    return json.dumps(obj)

In [59]:
gps_df = pd.read_csv(gps_inp)
gps_df['json'] = gps_df.apply(lambda u: to_json(u), axis=1)

In [60]:
gps_df.head()

,ts,index_record,heading,lat,lng,positionTime,h_positionTime,h_receiveTime,json
0,1622119844873,0,26,21.03023,105.78133,1622119835,2021-05-27 19:50:35,2021-05-27 19:50:44,"{""lat"": 21.03023, ""lng"": 105.78133000000001, ""..."
1,1622119844873,1,26,21.03023,105.78133,1622119836,2021-05-27 19:50:36,2021-05-27 19:50:44,"{""lat"": 21.03023, ""lng"": 105.78133000000001, ""..."
2,1622119844873,2,26,21.03023,105.78134,1622119837,2021-05-27 19:50:37,2021-05-27 19:50:44,"{""lat"": 21.03023, ""lng"": 105.78133999999999, ""..."
3,1622119844873,3,26,21.03024,105.78134,1622119838,2021-05-27 19:50:38,2021-05-27 19:50:44,"{""lat"": 21.03024, ""lng"": 105.78133999999999, ""..."
4,1622119844873,4,26,21.03023,105.78134,1622119839,2021-05-27 19:50:39,2021-05-27 19:50:44,"{""lat"": 21.03023, ""lng"": 105.78133999999999, ""..."


In [61]:
vehicle_df = pd.read_csv(vehicle_inp)
vehicle_df['json'] = vehicle_df.apply(lambda r: to_json(r, id), axis=1)
vehicle_df['type'] = 'vehicle'

### Merge

In [62]:
gps_merge = pd.DataFrame()
gps_merge = gps_df.groupby(by='ts').apply(lambda row: merge_gps(row, id))

gps_merge = gps_merge.reset_index()
gps_merge.columns = ['ts', 'json']
gps_merge['type'] = 'gps'

In [63]:
gps_merge['json'][0]

'{"id": "sample car 1", "samples": [{"lat": 21.03023, "lng": 105.78133000000001, "time": 1622119835000, "azimuth": 26}, {"lat": 21.03023, "lng": 105.78133000000001, "time": 1622119836000, "azimuth": 26}, {"lat": 21.03023, "lng": 105.78133999999999, "time": 1622119837000, "azimuth": 26}, {"lat": 21.03024, "lng": 105.78133999999999, "time": 1622119838000, "azimuth": 26}, {"lat": 21.03023, "lng": 105.78133999999999, "time": 1622119839000, "azimuth": 26}, {"lat": 21.03023, "lng": 105.78133999999999, "time": 1622119840000, "azimuth": 26}]}'

In [64]:
merge_all = pd.concat([gps_merge, vehicle_df[['ts', 'json', 'type']]], ignore_index=True)
merge_all.sort_values(by='ts', inplace=True)

In [65]:
for i in range(5):
    print merge_all.iloc[i]['ts']

1622119844873
1622119847071
1622119852895
1622119855490
1622119860957


In [ ]:
for i in range(merge_all.shape[0]):
    row = merge_all.iloc[i]
#     print row
    push = True
    if row['type'] == 'gps':
        url = tracking_gps_url
        push = True
    else:
        push = True
        url = tracking_vehicle_url
    
    if push:
        x = requests.post(url, data=row['json'])
        print(row['json'])
        print(x.text)
        time.sleep(delay)

{"id": "sample car 1", "samples": [{"lat": 21.03023, "lng": 105.78133000000001, "time": 1622119835000, "azimuth": 26}, {"lat": 21.03023, "lng": 105.78133000000001, "time": 1622119836000, "azimuth": 26}, {"lat": 21.03023, "lng": 105.78133999999999, "time": 1622119837000, "azimuth": 26}, {"lat": 21.03024, "lng": 105.78133999999999, "time": 1622119838000, "azimuth": 26}, {"lat": 21.03023, "lng": 105.78133999999999, "time": 1622119839000, "azimuth": 26}, {"lat": 21.03023, "lng": 105.78133999999999, "time": 1622119840000, "azimuth": 26}]}
"SUCCESS"
{"time": 1622119846000, "height": 24, "azimuth": 18, "lat": 21.03021, "velocity": 8, "lng": 105.7813, "id": "sample car 1"}
"SUCCESS"
{"id": "sample car 1", "samples": [{"lat": 21.03021, "lng": 105.7813, "time": 1622119845000, "azimuth": 18}, {"lat": 21.03021, "lng": 105.7813, "time": 1622119846000, "azimuth": 18}, {"lat": 21.03022, "lng": 105.7813, "time": 1622119847000, "azimuth": 38}, {"lat": 21.03022, "lng": 105.78132, "time": 1622119848000, 

{"time": 1622119937000, "height": 4, "azimuth": 8, "lat": 21.034010000000002, "velocity": 36, "lng": 105.78033, "id": "sample car 1"}
"SUCCESS"
{"id": "sample car 1", "samples": [{"lat": 21.03365, "lng": 105.78027, "time": 1622119933000, "azimuth": 9}, {"lat": 21.033739999999998, "lng": 105.78028, "time": 1622119934000, "azimuth": 9}, {"lat": 21.03383, "lng": 105.7803, "time": 1622119935000, "azimuth": 8}, {"lat": 21.033920000000002, "lng": 105.78031000000001, "time": 1622119936000, "azimuth": 8}, {"lat": 21.034010000000002, "lng": 105.78033, "time": 1622119937000, "azimuth": 8}, {"lat": 21.0341, "lng": 105.78034, "time": 1622119938000, "azimuth": 10}, {"lat": 21.03418, "lng": 105.78036000000002, "time": 1622119939000, "azimuth": 10}, {"lat": 21.034270000000003, "lng": 105.78036999999999, "time": 1622119940000, "azimuth": 11}]}
"SUCCESS"
{"time": 1622119945000, "height": 4, "azimuth": 9, "lat": 21.03471, "velocity": 36, "lng": 105.78045, "id": "sample car 1"}
"SUCCESS"
{"id": "sample c

{"id": "sample car 1", "samples": [{"lat": 21.03217, "lng": 105.77976000000001, "time": 1622120013000, "azimuth": 185}, {"lat": 21.03205, "lng": 105.77975, "time": 1622120014000, "azimuth": 184}, {"lat": 21.03193, "lng": 105.77973999999999, "time": 1622120015000, "azimuth": 185}, {"lat": 21.03181, "lng": 105.77972, "time": 1622120016000, "azimuth": 187}, {"lat": 21.03169, "lng": 105.7797, "time": 1622120017000, "azimuth": 188}, {"lat": 21.031570000000002, "lng": 105.77968, "time": 1622120018000, "azimuth": 190}, {"lat": 21.03145, "lng": 105.77965, "time": 1622120019000, "azimuth": 190}, {"lat": 21.03133, "lng": 105.77963000000001, "time": 1622120020000, "azimuth": 190}]}
"SUCCESS"
{"id": "sample car 1", "samples": [{"lat": 21.03121, "lng": 105.7796, "time": 1622120021000, "azimuth": 190}, {"lat": 21.03109, "lng": 105.77958999999998, "time": 1622120022000, "azimuth": 189}, {"lat": 21.03098, "lng": 105.77956, "time": 1622120023000, "azimuth": 190}, {"lat": 21.03086, "lng": 105.77954, "ti

{"id": "sample car 1", "samples": [{"lat": 21.02019, "lng": 105.78011000000001, "time": 1622120093000, "azimuth": 144}, {"lat": 21.02005, "lng": 105.78021000000001, "time": 1622120094000, "azimuth": 144}, {"lat": 21.0199, "lng": 105.78033, "time": 1622120095000, "azimuth": 143}, {"lat": 21.01976, "lng": 105.78043999999998, "time": 1622120096000, "azimuth": 143}, {"lat": 21.01962, "lng": 105.78056000000001, "time": 1622120097000, "azimuth": 142}, {"lat": 21.01949, "lng": 105.78067, "time": 1622120098000, "azimuth": 143}, {"lat": 21.01935, "lng": 105.78078000000001, "time": 1622120099000, "azimuth": 142}, {"lat": 21.01921, "lng": 105.7809, "time": 1622120100000, "azimuth": 143}]}
"SUCCESS"
{"time": 1622120104000, "height": 15, "azimuth": 143, "lat": 21.01867, "velocity": 69, "lng": 105.78133999999999, "id": "sample car 1"}
"SUCCESS"
{"id": "sample car 1", "samples": [{"lat": 21.019070000000003, "lng": 105.78101000000001, "time": 1622120101000, "azimuth": 142}, {"lat": 21.01894, "lng": 10